# Project-Management-Problem-Consulting

Plan:
- Tech company with 100 employees
- A project of delivering a cloud migration platform
- Planned to be completed in 1 year within $3M
- 200 tasks in the project plan

Current Situation:
- 9 months into the project
- Project is behind schedule
- Need 6 more months and $1M more to complete

## 1. Generate Example Data

In [1]:
import pandas as pd
import numpy as np
import random

random.seed(42)
np.random.seed(42)

NUM_TASKS = 200
TOTAL_BUDGET = 3_000_000
PROJECT_MONTHS = 12
ELAPSED_MONTHS = 9

# 100 employees:
all_employees = [f"Employee_{i+1}" for i in range(100)]

# 80 employees didn't complete their work on time:
delayed_employees = set(random.sample(all_employees, 80))
on_time_employees = set(all_employees) - delayed_employees

departments = ["Platform", "Security", "Data", "Networking"]


# Generate tasks

costs_random_weights = np.random.uniform(1000, 10000, NUM_TASKS)
planned_costs = TOTAL_BUDGET * costs_random_weights / costs_random_weights.sum()

tasks = []
for i in range(NUM_TASKS):
    department = random.choice(departments)
    owner = random.choice(all_employees)
    start_month = random.randint(1, PROJECT_MONTHS // 2)
    planned_duration_months = random.randint(1, 6)
    planned_end_month = min(start_month + planned_duration_months - 1, PROJECT_MONTHS)
    risk_level = random.choice(["Low", "Medium", "High"])
    planned_cost = round(float(planned_costs[i]), 2)

    # set delay:
    if owner in delayed_employees:
        delay = random.choices([1, 2, 3], weights=[0.3, 0.4, 0.3])[0]  # delay 1 - 3 months
    else:
        delay = random.choices([0, -1], weights=[0.8, 0.2])[0]  # on time or complete 1 month ahead

    actual_end_month = min(max(planned_end_month + delay, start_month), PROJECT_MONTHS)
    completed_on_time = actual_end_month <= planned_end_month

    tasks.append({
        "TaskID": i + 1,
        "TaskName": f"Task {i + 1}",
        "Department": department,
        "Owner": owner,
        "RiskLevel": risk_level,
        "StartMonth": start_month,
        "PlannedDurationMonths": planned_duration_months,
        "PlannedEndMonth": planned_end_month,
        "ActualEndMonth": actual_end_month,
        "PlannedCostUSD": planned_cost,
        "CompletedOnTime": completed_on_time
        })

tasks_df = pd.DataFrame(tasks)

# Generate records

records = []
for _, task in tasks_df.iterrows():
    active_months = list(range(task["StartMonth"], task["ActualEndMonth"] + 1))
    planned_cost_per_month = task["PlannedCostUSD"] / len(active_months)

    progress_random_weights = np.random.uniform(0.1, 1, len(active_months))
    months_progress = progress_random_weights / progress_random_weights.sum()
    
    cost_factors = np.random.uniform(0.9, 1.3, len(active_months))

    cumulative_progress = 0

    for month in range(1, ELAPSED_MONTHS + 1):

        # assign actual progress if the task is active in this month:
        if month in active_months:
            idx = active_months.index(month)
            actual_progress = months_progress[idx]
            spend = task["PlannedCostUSD"] * actual_progress * cost_factors[idx]
        else:
            actual_progress = 0
            spend = 0

        cumulative_progress = min(1, cumulative_progress + actual_progress)  # prevent progress from exceeding 1
        
        records.append({
            "TaskID": task["TaskID"],
            "TaskName": task["TaskName"],
            "Department": task["Department"],
            "Owner": task["Owner"],
            "RiskLevel": task["RiskLevel"],
            "StartMonth": task["StartMonth"],
            "PlannedDurationMonths": task["PlannedDurationMonths"],
            "PlannedEndMonth": task["PlannedEndMonth"],
            "ActualEndMonth": task["ActualEndMonth"],
            "PlannedCostUSD": task["PlannedCostUSD"],
            "Month": month,
            "MonthActualProgressPct": round(actual_progress * 100, 2),
            "MonthCumulativeProgressPct": round(cumulative_progress * 100, 2),
            "MonthSpendUSD": round(spend, 2),
            "CompletedOnTime": task["CompletedOnTime"]
        })

        # if already finished, then stop:
        if cumulative_progress >= 1:
            break

df = pd.DataFrame(records)
df.to_csv("data.csv", index=False)

Preview the data:

In [2]:
df.head(30)

,TaskID,TaskName,Department,Owner,RiskLevel,StartMonth,PlannedDurationMonths,PlannedEndMonth,ActualEndMonth,PlannedCostUSD,Month,MonthActualProgressPct,MonthCumulativeProgressPct,MonthSpendUSD,CompletedOnTime
0,1,Task 1,Platform,Employee_78,High,6,2,7,10,12240.89,1,0.00,0.00,0.00,False
1,1,Task 1,Platform,Employee_78,High,6,2,7,10,12240.89,2,0.00,0.00,0.00,False
2,1,Task 1,Platform,Employee_78,High,6,2,7,10,12240.89,3,0.00,0.00,0.00,False
3,1,Task 1,Platform,Employee_78,High,6,2,7,10,12240.89,4,0.00,0.00,0.00,False
4,1,Task 1,Platform,Employee_78,High,6,2,7,10,12240.89,5,0.00,0.00,0.00,False
5,1,Task 1,Platform,Employee_78,High,6,2,7,10,12240.89,6,25.54,25.54,2825.71,False
6,1,Task 1,Platform,Employee_78,High,6,2,7,10,12240.89,7,6.62,32.17,762.48,False
7,1,Task 1,Platform,Employee_78,High,6,2,7,10,12240.89,8,9.25,41.42,1319.65,False
8,1,Task 1,Platform,Employee_78,High,6,2,7,10,12240.89,9,34.25,75.66,3781.22,False
9,2,Task 2,Security,Employee_60,High,4,3,6,8,26763.43,1,0.00,0.00,0.00,False


## 2. Call OpenAI API

In [3]:
import openai
from dotenv import load_dotenv

In [4]:
system_instructions = """You are a professional business management consultant at a consulting firm, specializing in project management, risk assessment, and process optimization. Your role is to analyze the provided data, identify underlying problems or inefficiencies, and deliver concise, well-structured, and practical solutions and recommendations based on your findings. Cite specific data when necessary.
"""

In [5]:
with open("data.csv", "r") as f:
    data_text = f.read()

user_input = f"""We are a tech company with 100 employees. We are working on a project of delivering a cloud migration platform. It has 200 tasks. We plan to have it completed in 1 year under budget $3 million USD.

Currently, we are 9 months into the project and the project is behind the schedule. We estimate that we probably need 6 additional months and extra $1 million USD to complete.

We have identified the following issues with our team:
1. The task management is manually updated every week by the project manager. 
2. The status reports are manually produced and sent by the project manager to the CTO once a week.
3. 20 of the employees did not complete their tasks on time which affects the remaining 80 employees.

Below is the working data of the project in csv format. Please identify all potential problems and provide a detailed solution to the problems.

{data_text}
"""

In [6]:
load_dotenv()

client = openai.OpenAI()

response = client.responses.create(
    model="gpt-4o-mini",
    instructions=system_instructions,
    input=user_input
)

print(response.output_text)

### Analysis of Current Project Status

#### Current Project Status:
- **Timeline**: Originally planned for 12 months, now estimated to require an additional 6 months (totaling 18 months).
- **Budget**: Originally set at $3 million; expected to exceed by an additional $1 million.
- **Total Task Count**: 200 tasks, with 20 employees failing to complete tasks on time, causing delays for the remaining 80 employees.

#### Identified Problems:
1. **Manual Task Management**:
   - The project manager manually updates task statuses weekly, which can lead to inefficiencies and errors in communication and tracking.
  
2. **Manual Status Reporting**:
   - Weekly status reports are manually prepared and sent to the CTO, which can result in delays in addressing issues that arise.

3. **Delayed Tasks**:
   - 20 employees did not complete their tasks on time, causing a bottleneck and delays for others.
  
4. **High Risk Tasks**:
   - Many tasks are categorized as 'High' risk, indicating potential iss

In [7]:
print(response.usage)

ResponseUsage(input_tokens=65788, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=829, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=66617)
